In [1]:
import pandas as pd
import os
import scipy.stats as stats

In [2]:
carpetas = ['GUROBI','MLM']

In [3]:
gur = []
mlm = []

for c in carpetas:
    archivos = os.listdir(c)
    for ar in archivos: 
        tiempo, solver, capacidad, nodos  = sorted(ar.rstrip(".xlsx").split("_"))
        tiempo = int(tiempo)
        capacidad = int(capacidad[1:])
        nodos = int(nodos[1:])
        datos = pd.read_excel(f"{c}/{ar}")
        datos["capacidad"] = capacidad
        datos["nodos"] = nodos
        datos["tiempos"] = tiempo
        if solver == "GUROBI":
            gur.append(datos)
        elif solver == "MLM":
            mlm.append(datos)
        else: pass
        
gurobis = pd.concat(gur)
mlms    = pd.concat(mlm)

In [4]:
bks = pd.concat((
    mlms.groupby(["name", "capacidad", "nodos"]).agg(bks = ("min", "min")).reset_index(), 
    gurobis.groupby(["name", "capacidad", "nodos"]).agg(bks = ("UB", "min")).reset_index()
))
bks = bks.groupby(["name", "capacidad", "nodos"]).agg(bks = ("bks", "min")).reset_index()

gurobis = gurobis.merge(bks, on = ["name", "capacidad", "nodos"])
mlms    = mlms.merge(bks, on = ["name", "capacidad", "nodos"])

In [5]:
gurobis["best"] = gurobis["UB"]
mlms["best"] = mlms["min"]

gurobis["gap_best"] = (gurobis["UB"] - gurobis["bks"]) / gurobis["bks"]
mlms["gap_best"] = (mlms["min"] - mlms["bks"]) / mlms["bks"]
mlms["gap_avg"] = (mlms["avg"] - mlms["bks"]) / mlms["bks"]


In [6]:
def wilcoxon(gurobis, mlms, base = "gap_best", 
             tiempos = [60, 120, 180], nodos = [100, 150, 200]):
    global exp1, exp2, exp3
    tabla = []
    for n in nodos:
        fila = dict()
        fila["tamaño"] = n
        for t in tiempos:
            exp1 = mlms[(mlms.tiempos == t) & (mlms.nodos == n)][base]
            exp2 = gurobis[(gurobis.tiempos == t) & (gurobis.nodos == n)][base]
            fila[f"gurobi {t}"] = stats.wilcoxon(exp1, exp2, alternative = "less").pvalue
            try:
                exp3 = gurobis[(gurobis.tiempos == 3600) & (gurobis.nodos == n)][base]
                fila[f"gurobi benckmark-{t}"] = stats.wilcoxon(exp1, exp3, alternative = "less").pvalue
            except: pass
        tabla.append(fila)
    return pd.DataFrame(tabla)
    
tabla = wilcoxon(gurobis, mlms)
tabla = tabla[sorted(tabla.columns)]

In [7]:
tabla2 = tabla.set_index("tamaño")
tabla2

,gurobi 120,gurobi 180,gurobi 60,gurobi benckmark-120,gurobi benckmark-180,gurobi benckmark-60
tamaño,,,,,,
100,NaN,NaN,2.056202e-08,NaN,NaN,1.0
150,2.233470e-37,1.529512e-34,4.101050e-14,1.0,1.0,1.0
200,2.487865e-42,4.106136e-24,1.123184e-10,1.0,1.0,1.0


In [8]:
def wilcoxon2(gurobis, mlms, base = "gap_best", 
             tiempos = [60, 120, 180], nodos = [100, 150, 200]):
    global exp1, exp2, exp3
    tabla = []
    for n in nodos:
        fila = dict()
        fila["tamaño"] = n
        for t in tiempos:
            exp1 = mlms[(mlms.tiempos == t) & (mlms.nodos == n)][base]
            exp2 = gurobis[(gurobis.tiempos == t) & (gurobis.nodos == n)][base]
            fila[f"gurobi {t}"] = stats.wilcoxon(exp1, exp2, alternative = "less").pvalue
        tabla.append(fila)
    return pd.DataFrame(tabla)
    
tabla = wilcoxon2(gurobis, mlms)
tabla = tabla[sorted(tabla.columns)]

tabla2 = tabla.set_index("tamaño")
tabla2

,gurobi 120,gurobi 180,gurobi 60
tamaño,,,
100,NaN,NaN,2.056202e-08
150,2.233470e-37,1.529512e-34,4.101050e-14
200,2.487865e-42,4.106136e-24,1.123184e-10
